In [1]:
import json
import openai
import requests
from flujo_de_dialogo import HistoricoConversacion
from calcular_tokens import num_tokens_from_messages, num_tokens_from_string
from open_ai_utils import simular_respuesta_generativa
from nlp2SQL import LLMsToDataBase
from nlp2SQL import SQLAgent

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

import os
from dotenv import load_dotenv
from load_store_utils import cargar_lote_datos
from open_ai_utils import enviar_promt_chat_completions_mode
import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [2]:
'dame los nombres de los actores y actrizes que hayan participado en al menos dos peliculas de más de 181 minutos'

'dame los nombres de los actores y actrizes que hayan participado en al menos dos peliculas de más de 181 minutos'

In [4]:
finalizar = False
primera_consulta = True

pagila_bbdd = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

while finalizar == False:

    conversacion = True
    nueva_consulta = True

    ### podria hacer una funcion para reconocer a que base de datos se quiere conectar el usuario

    # el usuario hace la consulta
    simular_respuesta_generativa('AGENTE:\nSoy tu agente experto en bases de datos. ¿En que te puedo ayudar hoy?\n\n')
    consulta_usuario = str(input())

    # inicializa el agente SOLO en la primera consulta
    if primera_consulta:
        # se establece la conexion y se crea el agente
        llm_to_pagila_conection = LLMsToDataBase(**pagila_bbdd)
        agent = SQLAgent(llm_to_database=llm_to_pagila_conection)
        primera_consulta= False
        

    while conversacion: 
        
        # gestionar la consulta si es una consulta nueva
        if nueva_consulta==True: 

            # printeamos la consulta del usuario
            simular_respuesta_generativa(f'\nPROMPT_USUARIO_{agent.historico.contador_interacciones}:\n{consulta_usuario}\n\n')

            # le pedimos la consulta al agente y printeamos la tabla
            simular_respuesta_generativa('\nAGENTE:\n')
            tabla_resultado, codigo_sql = agent.nlp_to_sql(
                consulta_nlp=consulta_usuario, 
                metadata_token_limit= 1000, 
                        )      

            simular_respuesta_generativa(f'\nAGENTE:\nAqui tienes el resultado de tu consulta:\n{tabla_resultado}\n\n')

            # generamos el informe de la consulta dandole al agente la tabla el codigo sql y la consulta del usuario
            simular_respuesta_generativa('\nAGENTE:\nMe dispongo a generar el informe de tu consulta...\n\n')
            informe = agent.informe_resultado(
                consulta_usuario= consulta_usuario, 
                tabla_texto=tabla_resultado, 
                max_tokens_respuesta=1000, 
                codigo_sql= codigo_sql )
            
            # printeamos el informe
            simular_respuesta_generativa(f'\nAGENTE:\nAqui tienes el informe de tu consulta: \n{informe}\n\n')

            # actualizar historico
            agent.historico.guardar_historico(
                mensaje_usuario= consulta_usuario, 
                respuesta_sistema= informe
                )


        # continuamos chat sobre la lectura o nueva consulta
        simular_respuesta_generativa('\nAGENTE:\n¿Tienes alguna otra consulta?\n\n')
        consulta_usuario = str(input())
        conversacion, nueva_consulta = agent.continuar_conversando(
            usuario= consulta_usuario, 
            historico= agent.historico.ventana_historico_final(), 
            tabla_consulta_anterior=tabla_resultado)

        while conversacion == True and nueva_consulta== False:

            # printeamos la consulta del usuario sobre respuestas anteriores
            simular_respuesta_generativa(f'\nPROMPT_USUARIO_{agent.historico.contador_interacciones}:\n{consulta_usuario}\n\n')

            respuesta_agente = agent.pregunta_sobre_consulta_anterior(
                usuario= consulta_usuario, 
                ultimo_historico=agent.historico.ventana_historico_final(), 
                tabla_consulta_anterior=tabla_resultado
                )
            # printeamos las nuevas respuesta del sistema
            simular_respuesta_generativa(f'\nAGENTE:\nLa respuesta a tu consulta:\n{respuesta_agente}\n\n')

            # continuamos chat sobre la lectura o nueva consulta
            simular_respuesta_generativa('\nAGENTE:\n¿Tienes alguna otra consulta?\n\n')
            consulta_usuario = str(input())
            conversacion, nueva_consulta = agent.continuar_conversando(
                usuario= consulta_usuario, 
                historico= agent.historico.ventana_historico_final()
                )
            
            # actualizar historico
            agent.historico.guardar_historico(
                mensaje_usuario=consulta_usuario, 
                respuesta_sistema=respuesta_agente
                )
            
    simular_respuesta_generativa('AGENTE:\nEstas seguro de que quieres cerrar el chat? (Y/N)\n\n')
    ultima_oportunidad = str(input())

    if ultima_oportunidad.lower() == 'y': 
        finalizar = True
        agent.close_connection()

simular_respuesta_generativa(f'\n\nHa sido un placer ayudarte. Hasta la próxima!!')

AGENTE:
Soy tu agente experto en bases de datos. ¿En que te puedo ayudar hoy?


PROMPT_USUARIO_1:
dame los nombres de los actores y actrizes que hayan participado en al menos dos peliculas de más de 181 minutos


AGENTE:

Voy a asegurar que los metadatos de bbdd pagila esten disponibles...

El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.


El codigo sql que se ejecutará para responder tu consulta:
SELECT a.first_name, a.last_name, COUNT(fa.film_id) AS number_of_films
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.length > 181
GROUP BY a.actor_id
HAVING COUNT(fa.film_id) >= 2;


AGENTE:
Aqui tienes el resultado de tu consulta:
|    | first_name   | last_name    |   number_of_films |
|---:|:-------------|:-------------|------------------:|
|  0 | Mena         | Hopper   

TypeError: cannot unpack non-iterable NoneType object

In [5]:
print(agent.historico.historico)

prompt USUARIO_1:
dame los nombres de los actores y actrizes que hayan participado en al menos dos peliculas de más de 181 minutos

Respuesta AGENTE_1:
Los actores y actrices que han participado en al menos dos películas de más de 181 minutos son:

1. Mena Hopper
2. Jeff Silverstone
3. Fay Wood
4. Chris Bridges
5. Liza Bergman
6. Minnie Kilmer
7. Cuba Olivier
8. Rip Crawford
9. Daryl Wahlberg
10. Jude Cruise
11. Bob Fawcett
12. Mary Keitel
13. Woody Hoffman
14. Johnny Lollobrigida
15. Matthew Carrey
16. Ben Willis
17. Sean Guiness
18. Reese Kilmer
19. Gina Degeneres
20. Christopher West
21. Charlize Dench
22. William Hackman
23. Burt Dukakis
24. Uma Wood
25. Cuba Allen
26. Jayne Silverstone
27. Jayne Neeson
28. Cate Mcqueen
29. Meryl Allen
30. Jodie Degeneres
31. Parker Goldberg
32. Spencer Depp
33. River Dean
34. Sandra Kilmer
35. Christian Akroyd

Cabe destacar que Mena Hopper, Mary Keitel, River Dean han participado en cuatro películas que cumplen con el criterio, mientras que Chris